# Projet MALAP

## Réalisé par Tong Zhao, Quentin Duchemin et Pierre Boyeau

In [13]:
import  numpy as np
import matplotlib.pyplot as plt
from tools import *
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
from scipy import stats
from sklearn.svm import SVC

In [14]:
def load_usps(filename):
    with open(filename,"r") as f:
        f.readline()
        data =[ [float(x) for x in l.split()] for l in f if len(l.split())>2]
    tmp = np.array(data)
    return tmp[:,1:],tmp[:,0].astype(int)

def get_usps(l,datax,datay):
    """ l : liste des chiffres a extraire"""
    if type(l)!=list:
        resx = datax[datay==l,:]
        resy = datay[datay==l]
        return resx,resy
    tmp =  list( zip(*[get_usps(i,datax,datay) for i in l]))
    tmpx,tmpy = np.vstack(tmp[0]),np.hstack(tmp[1])
    idx = np.random.permutation(range(len(tmpy)))
    return tmpx[idx,:],tmpy[idx]

def show_usps(data):
    plt.imshow(data.reshape((16,16)),interpolation="nearest",cmap="gray")

## Data processing

In [157]:
# Définition de la base d'apprentissage
xuspstrain,yuspstrain = load_usps("datas/usps/USPS_train.txt")
xuspstrain,yuspstrain = get_usps([5, 8],xuspstrain,yuspstrain)
data = xuspstrain
(n,m) = np.shape(data)

# shuffle data
idx = np.random.permutation(n)
data = data[idx]
yuspstrain = yuspstrain[idx]

# Pourcentage de la base d'apprentissage non étiquettée
percent_u = 95./100.
# nombre de données non étiquettées
U = int(percent_u * n)
# nombre de données étiquettées
L =  n-U

# étiquettes connues
labels = yuspstrain[:L]
# étiquettes à prédire
labpredire = yuspstrain[L:]

print("Load data...\n")
print("Data shape: %d * %d" % data.shape)
print("\nLabeled num: %d" % L)
print("\nUnlabeld num: %d" % U)

Load data...

Data shape: 1098 * 256

Labeled num: 55

Unlabeld num: 1043


## Baseline 1 - K plus proche voisins 

In [16]:
class KNN:
    
    def __init__(self, k):
        self.k = k
        
    def fit(self, data, labels):
        self.N = data.shape[0]
        L = labels.shape[0]
        
        # calculate distance
        self.W = cosine_distances(data[L:], data[:L])
        self.labels = labels
        
    def predict(self):
        
        idx = np.argsort(self.W, axis = 1)[:, :self.k]
        bag = self.labels[idx]
        predict, _ = stats.mode(bag, axis = 1)
        
        return predict.reshape((-1))
    
    def score(self, labels):
        return (self.predict()==labels).mean()

In [158]:
M = KNN(10)
M.fit(data,labels)
print("The score of Diffusion by knn is %f" % M.score(labpredire))

The score of Diffusion by knn is 0.934803


## Baseline 2 - SVM

In [159]:
clf = SVC(C=1.5)
clf.fit(data[:L], labels)
print("The score of SVM is %f" % (clf.predict(data[L:]) == labpredire).mean())

The score of SVM is 0.970278


## 1. Classification using simply threshold

$$ f_u = (D_{uu} - W_{uu})^{-1} W_{ul} f_l$$

In [103]:
class DiffusionTRESH:
        
    def predict(self):
        
        # record label
        set_labels = np.unique(self.labels)

        y = (np.tile(self.labels.reshape((-1, 1)), (1, set_labels.shape[0])) == set_labels).astype(int)
        fu = np.dot(np.dot(np.linalg.inv(self.D[self.L:,self.L:] - self.W[self.L:,self.L:]),self.W[self.L:,:self.L]),y)
        prediction = set_labels[np.argmax(fu, axis = 1)]
        return prediction          
            
    def fit(self, data, labels):
        self.labels = labels
        self.L = labels.shape[0]
        sigmas = np.array([2.5 for i in range(m)])
        
        # calculate W
        data_n = data / sigmas
        self.W = np.exp(-euclidean_distances(data_n)**2)
        
        # calculate D
        diago = np.sum(self.W,axis=1)
        self.D = np.diag(diago)
        
        
    def score(self, labels):
        return (self.predict()==labels).mean()


In [160]:
M = DiffusionTRESH()
M.fit(data,labels)
print("The score of Diffusion by threshold is %f" % M.score(labpredire))

The score of Diffusion by threshold is 0.986577



## 2. Incorporation of Class Prior : CMN with weights fixed by advance

In [23]:
class DiffusionCMN:

    def predict(self):
        
        # record label
        set_labels = np.unique(self.labels)
        
        # calculate fu
        y = (np.tile(self.labels.reshape((-1, 1)), (1, set_labels.shape[0])) == set_labels).astype(int)
        desirable_proportions = y.sum(0) + 1
        fu = np.dot(np.dot(np.linalg.inv(self.D[self.L:,self.L:]-self.W[self.L:,self.L:]),self.W[self.L:,:self.L]),y)
        fu = fu * (desirable_proportions / fu.sum(0))

        prediction = set_labels[np.argmax(fu, axis = 1)]
        return prediction          
            
    def fit(self,data,labels):
        self.labels = labels
        self.L = labels.shape[0]
        sigmas = np.array([2.5 for i in range(m)])
        
        # calculate W
        data_n = data / sigmas
        self.W = np.exp(-euclidean_distances(data_n)**2)
        
        # calculate D
        diago = np.sum(self.W,axis=1)
        self.D = np.diag(diago)
        
    def score(self,labels):
        return (self.predict()==labels).mean()

In [161]:
M = DiffusionCMN()
M.fit(data,labels)
print("The score of Diffusion by class prior is %f" % M.score(labpredire))

The score of Diffusion by class prior is 0.985618


## 3. Incorporating External Classifiers - SVM

In [25]:
class DiffusionSVM:

    def predict(self):
        
        # record label
        set_labels = np.unique(self.labels)
        
        # calculate fu
        y = (np.tile(self.labels.reshape((-1, 1)), (1, set_labels.shape[0])) == set_labels).astype(int)
        fu = np.dot(np.dot(np.linalg.inv(self.D[self.L:,self.L:]-self.W[self.L:,self.L:]),self.W[self.L:,:self.L]),y)
        fu = fu * self.hu
        
        prediction = set_labels[np.argmax(fu, axis = 1)]
        return prediction          
            
    def fit(self,data,labels):
        self.labels = labels
        self.L = labels.shape[0]
        sigmas = np.array([2.5 for i in range(m)])
        
        # calculate W
        data_n = data / sigmas
        self.W = np.exp(-euclidean_distances(data_n)**2)
        
        # calculate D
        diago = np.sum(self.W,axis=1)
        self.D = np.diag(diago)
        
        # train svm
        clf = SVC(C=2, probability = True)
        clf.fit(data[:self.L], labels)
        self.hu = clf.predict_proba(data[L:])
        
    def score(self,labels):
        return (self.predict()==labels).mean()

In [162]:
M = DiffusionSVM()
M.fit(data,labels)
print("The score of Diffusion by class prior is %f" % M.score(labpredire))

The score of Diffusion by class prior is 0.989453


# 4. Learning W

In [155]:
class DiffusionLEARN:
    
    def __init__(self, eps=0.01, lr = 0.1):        
        self.eps = eps
        self.lr = lr

    def oracle(self, sgm, max_iter):
        
        self.h_histo = [1000]
        self.dh_histo = None
        
        for i in range(max_iter):
            # calculate W
            data_sgm = self.data / sgm
            self.W = np.exp(-euclidean_distances(data_sgm) ** 2)
            # calculate D
            diago = np.sum(self.W,axis=1)
            self.D = np.diag(diago)
            # calculate P
            P = np.dot(np.linalg.inv(self.D), self.W)
            # calculate fu
            fu = np.dot(np.dot(np.linalg.inv(np.eye(self.U) - P[L:, L:]), P[L:, :L]), self.fl)
            # calculate H
            self.h_histo.append(self.H(fu))
            # if this is the right direction
            if self.h_histo[-1] <= self.h_histo[-2]:
                # calculate the gradient of H
                dh = self.gradH(fu, P, sgm)
                self.dh_histo = np.hstack((self.dh_histo, dh)) if self.dh_histo != None else dh
                if np.all(np.abs(dh) < 1e-3) or self.lr < 1e-8:
                    break
                else:
                    sgm = sgm - self.lr * dh.reshape((1, -1))
            else:
                self.lr = self.lr / 5
                
        self.sigmas = sgm
        self.fu = fu

    def H(self, fu):
        return - (fu * np.log(fu)).sum() / self.U

    def gradH(self, fu, P, sgm):
        
        Ps = (1 - self.eps) * P + self.eps / (self.L + self.U)
        grad_h_sgm = []
        
        for i in range(256):
            Xi = np.tile(self.data[:, i], (self.U + self.L, 1))
            dw_sgm = 2 * self.W * ((Xi.T - Xi) ** 2) / (sgm[0, i] ** 3)
            # formule 14
            sum_dw = dw_sgm.sum(1).reshape((-1, 1))
            sum_w = self.W.sum(1).reshape((-1, 1))
            dps_sgm = (1 - self.eps) * (dw_sgm - P * sum_dw) / sum_w
            # formule 13
            dfu_sgm_1 = np.linalg.inv(np.eye(self.U) - Ps[self.L:, self.L:])
            dfu_sgm_2 = dps_sgm[self.L:, self.L:].dot(fu) + dps_sgm[self.L:, :self.L].dot(self.fl)
            dfu_sgm = np.dot(dfu_sgm_1, dfu_sgm_2)
            # formule 12
            dh_sgm = (fu[:,0] / fu[:, 1] * dfu_sgm[:, 1]).sum() / self.U
            # save result
            grad_h_sgm.append(dh_sgm)
            
        return np.array(grad_h_sgm).reshape((-1, 1))

    def fit(self, data, labels):
        self.labels = labels
        self.data = data
        self.L = labels.shape[0]
        self.U = data.shape[0] - self.L
        self.D = data.shape[1]
        
    def predict(self):
        
        # record label
        set_labels = np.unique(self.labels)
        # calculate fl
        self.fl = (np.tile(self.labels.reshape((-1, 1)), (1, set_labels.shape[0])) == set_labels).astype(int)
        sgm = np.array([2.5 for i in range(m)]).reshape((1, -1))
        # optimisation
        self.oracle(sgm, 5)
        # predict
        prediction = set_labels[np.argmax(self.fu, axis = 1)]
        return prediction          
    
    def score(self, labels):
        return (self.predict()==labels).mean()

In [163]:
M = DiffusionLEARN()
M.fit(data,labels)
print("The score of Diffusion by learning w is %f" % M.score(labpredire))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:29: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


The score of Diffusion by learning w is 0.985618
